LaBSE 
*   Paper: https://arxiv.org/pdf/2007.01852.pdf
*   Blog: https://ai.googleblog.com/2020/08/language-agnostic-bert-sentence.html 
*   Hugging face: https://huggingface.co/sentence-transformers/LaBSE

BIRCH


*   Paper: https://arxiv.org/abs/1903.10972
*   Code Repository: https://github.com/castorini/birch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/New_Work/Complete_Concept_Dataset.zip -d /content/
!unzip /content/drive/MyDrive/New_Work/Complete_Test_Collection_without_RBR.zip -d /content/

In [ ]:
import shutil
shutil.rmtree('/content/Train_Dataset/bangladesh')
shutil.rmtree('/content/Train_Dataset/economy')
shutil.rmtree('/content/Train_Dataset/education')
shutil.rmtree('/content/Train_Dataset/entertainment')
# shutil.rmtree('/content/Train_Dataset/health')
shutil.rmtree('/content/Train_Dataset/international')
shutil.rmtree('/content/Train_Dataset/kolkata')
shutil.rmtree('/content/Train_Dataset/life-style')
shutil.rmtree('/content/Train_Dataset/national')
shutil.rmtree('/content/Train_Dataset/opinion')
shutil.rmtree('/content/Train_Dataset/sports')
shutil.rmtree('/content/Train_Dataset/technology')
shutil.rmtree('/content/Train_Dataset/travel')
shutil.rmtree('/content/Train_Dataset/we-are')
shutil.rmtree('/content/Concept_Dataset/Old_Dataset_Concepts')
shutil.rmtree('/content/Concept_Dataset/New_Dataset_Concepts/Entertainment')
shutil.rmtree('/content/Concept_Dataset/New_Dataset_Concepts/International')
# shutil.rmtree('/content/Concept_Dataset/New_Dataset_Concepts/health')
shutil.rmtree('/content/Concept_Dataset/New_Dataset_Concepts/kolkata')
shutil.rmtree('/content/Concept_Dataset/New_Dataset_Concepts/national')
shutil.rmtree('/content/Concept_Dataset/New_Dataset_Concepts/sports')
shutil.rmtree('/content/Concept_Dataset/New_Dataset_Concepts/travel')

In [ ]:
concept_dir_path = "/content/Concept_Dataset/New_Dataset_Concepts/health"

In [ ]:
!pip install -U sentence-transformers
!pip install weighted-levenshtein
!pip install bnunicodenormalizer
!pip install indic-nlp-library
!pip install stopwordsiso
!pip install nltk
!pip install -U bnlp_toolkit
!pip install git+https://github.com/riteshpanjwani/pyiwn@master#egg=pyiwn
!pip install xlrd
!pip install XlsxWriter

In [ ]:
# # download tokenizer
!wget https://storage.googleapis.com/ai4b-public-data/indicBERT_sentencepiece_tokenizer.py

In [ ]:
import stopwordsiso as stopwords
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as eng_stopwords
import stopwordsiso as stopwords
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from bnlp import NLTKTokenizer
import sklearn
import numpy as np
from numpy.linalg import norm
import scipy

import ast
import xlsxwriter
import statistics

from indicBERT_sentencepiece_tokenizer import IndicBERTSentencePieceTokenizer
from indicnlp.tokenize import sentence_tokenize

import json
import pickle
from collections import defaultdict
from copy import copy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...


punkt not found. downloading...


[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
tr_file_paths=[]
tr_docs=[]
tr_dids=[]
factory = IndicNormalizerFactory()
normalizer = factory.get_normalizer("bn", remove_nuktas=False)
for dirpath, dirnames, filenames in os.walk("/content/Train_Dataset"):
  for file_name in filenames:
    file_path = os.path.join(str(dirpath), str(file_name))
    tr_file_paths.append(file_path)
    fileObject = open(file_path, "r+", encoding='utf-8', errors='ignore')
    text = fileObject.read()
    text = normalizer.normalize(text)
    tr_docs.append(text)
    doc_num = int(file_path.split('/')[-1].replace('.txt',''))
    tr_dids.append(doc_num)
print("Number of documents in the corpus: ",len(tr_file_paths))

In [ ]:
#@title Code to read ESA Concept dataset
c_docs = []
c_dids = []
c_file_paths = []
for dirpath, dirnames, filenames in os.walk(concept_dir_path):
  for file_name in filenames:
    # Get the absolute file path
    file_path = os.path.join(str(dirpath), str(file_name))
    c_file_paths.append(file_path)
    # Create a file object.
    fileObject = open(file_path, "r+", encoding='utf-8', errors='ignore')
    # Extract the text from the file.
    text = fileObject.read()
    c_docs.append(text)
    # using re.findall() we extract all the numbers from the string
    temp = re.findall(r'\d+', file_name)
    c_doc_num = ''.join(temp)
    c_dids.append(c_doc_num)

print("Number of documents in the corpus: ",len(c_dids))

In [ ]:
class document_linearization:
	def pre_process1(self,input_text):
		"""
		Description: Performs text normalisation, removes punctuations and stop words and returns a list of tokens
		:param input_text:  The text which needs to be cleaned.
		:param stop_words: List of stopwords to be removed from the input text.
		:return: list of tokens
		"""
		stop_words = stopwords.stopwords("bn").union(set(eng_stopwords.words('english')),
                                                     {'একটা', 'তাঁহার', 'দিয়া', 'বলিয়া', 'লইয়া', 'আসিয়া', 'লাগিল',
                                                      'কহিল', 'আসিয়া', 'তাহাকে', 'করিল', 'কহিলেন', 'কোনটি', 'ধরনের',
                                                      'বলিলেন', 'করিত', 'চলিয়া', 'ধরিয়া', 'পড়িয়া', 'করিবার', 'হইত',
                                                      'নহে', 'ফিরিয়া', 'কেহ', 'বলিতে', 'রহিল', 'পড়িল', 'উঠিল',
                                                      'বাহির', 'দেখিয়া', 'উঠিয়া', 'ফিরিয়া', 'অবশেষে', 'কিছুতেই',
                                                      'তেমনি', 'তোমাকে', 'নে', 'মাঝে', 'সকল', 'অত্যন্ত', 'একটু',
                                                      'একেবারে', 'এক', 'একটা', 'একদিন', 'কথা', 'তারই', 'হয়ে', 'হওয়ায়',
                                                      'রয়েছে', 'এমনকি', 'একজন', 'দিয়েছেন', 'একটা', 'হয়নি', 'সাথে',
                                                      'হয়েই', 'দিয়ে', 'কেমনে', 'করিয়ে', 'তোরা', 'জন্যে', 'পেয়ে',
                                                      'পাওয়া', 'তোর', 'ছাড়া', 'ছাড়াও', 'হওয়ার', 'তোমাদের', 'চেয়ে',
                                                      'কথা', 'জানিয়েছে', 'মত', 'অর্থাৎ', 'গিয়েছে', 'জাানিয়ে', 'হয়েছে',
                                                      'হিসেবে', 'হওয়া', 'এলো', 'করায়', 'তাঁহারা', 'দেওয়ার', 'হইয়া',
                                                      'হয়েছেন', 'তোদের', 'অর্ধভাগে', 'তিনই', 'এসো', 'দেয়', 'এক', 'যায়',
                                                      'দিয়েছে', 'চায়', 'হয়েছিল', 'তুই', 'হয়তো', 'হৈতে', 'অনুযায়ী',
                                                      'কয়েকটি', 'পাঁচ', 'করিয়া', 'সময়', 'থাকায়'})

		# Perform text normalisation
		# Reference: https://nbviewer.jupyter.org/url/anoopkunchukuttan.github.io/indic_nlp_library/doc/indic_nlp_examples.ipynb#Text-Normalization
		factory = IndicNormalizerFactory()
		normalizer = factory.get_normalizer("bn", remove_nuktas=False)
		output_text=normalizer.normalize(input_text)

		# Remove punctuation from the text.
		text = output_text.strip()
		text = text.replace("’", "")
		text = text.replace("‘", "")
		text = text.replace('”', "")
		text = text.replace("“", "")

		# Some of the punctuation symbols were obtained from here: https://github.com/paul-pias/Text-Preprocessing-in-Bangla-and-English
		clean_text = re.sub(r"[!\"”#$%&’()*+,-./:;<=>?@\[\]^_`{|}~–—\'।॥]+\ *", ' ', text)
		clean_text = re.sub(r"[-—]+\ *", ' ', clean_text)
		clean_text = re.sub(r"\n", '', clean_text)
		clean_text = " ".join(clean_text.split())

		# tokenize the cleaned text
		terms = clean_text.split(' ')
		# Remove stop words
		filtered_sentence = []
		for w in terms:
			# if w not in stop_words:
				filtered_sentence.append(w)
		terms = filtered_sentence
		return terms
	
	def pre_process(self,input_text):
		"""
		Description: Performs text normalisation, removes punctuations and stop words and returns a list of tokens
		:param input_text:  The text which needs to be cleaned.
		:param stop_words: List of stopwords to be removed from the input text.
		:return: list of tokens
		"""
		stop_words = stopwords.stopwords("bn").union(set(eng_stopwords.words('english')),
                                                     {'একটা', 'তাঁহার', 'দিয়া', 'বলিয়া', 'লইয়া', 'আসিয়া', 'লাগিল',
                                                      'কহিল', 'আসিয়া', 'তাহাকে', 'করিল', 'কহিলেন', 'কোনটি', 'ধরনের',
                                                      'বলিলেন', 'করিত', 'চলিয়া', 'ধরিয়া', 'পড়িয়া', 'করিবার', 'হইত',
                                                      'নহে', 'ফিরিয়া', 'কেহ', 'বলিতে', 'রহিল', 'পড়িল', 'উঠিল',
                                                      'বাহির', 'দেখিয়া', 'উঠিয়া', 'ফিরিয়া', 'অবশেষে', 'কিছুতেই',
                                                      'তেমনি', 'তোমাকে', 'নে', 'মাঝে', 'সকল', 'অত্যন্ত', 'একটু',
                                                      'একেবারে', 'এক', 'একটা', 'একদিন', 'কথা', 'তারই', 'হয়ে', 'হওয়ায়',
                                                      'রয়েছে', 'এমনকি', 'একজন', 'দিয়েছেন', 'একটা', 'হয়নি', 'সাথে',
                                                      'হয়েই', 'দিয়ে', 'কেমনে', 'করিয়ে', 'তোরা', 'জন্যে', 'পেয়ে',
                                                      'পাওয়া', 'তোর', 'ছাড়া', 'ছাড়াও', 'হওয়ার', 'তোমাদের', 'চেয়ে',
                                                      'কথা', 'জানিয়েছে', 'মত', 'অর্থাৎ', 'গিয়েছে', 'জাানিয়ে', 'হয়েছে',
                                                      'হিসেবে', 'হওয়া', 'এলো', 'করায়', 'তাঁহারা', 'দেওয়ার', 'হইয়া',
                                                      'হয়েছেন', 'তোদের', 'অর্ধভাগে', 'তিনই', 'এসো', 'দেয়', 'এক', 'যায়',
                                                      'দিয়েছে', 'চায়', 'হয়েছিল', 'তুই', 'হয়তো', 'হৈতে', 'অনুযায়ী',
                                                      'কয়েকটি', 'পাঁচ', 'করিয়া', 'সময়', 'থাকায়'})

		# Perform text normalisation
		# Reference: https://nbviewer.jupyter.org/url/anoopkunchukuttan.github.io/indic_nlp_library/doc/indic_nlp_examples.ipynb#Text-Normalization
		factory = IndicNormalizerFactory()
		normalizer = factory.get_normalizer("bn", remove_nuktas=False)
		output_text=normalizer.normalize(input_text)

		# Remove punctuation from the text.
		text = output_text.strip()
		text = text.replace("’", "")
		text = text.replace("‘", "")
		text = text.replace('”', "")
		text = text.replace("“", "")

		# Some of the punctuation symbols were obtained from here: https://github.com/paul-pias/Text-Preprocessing-in-Bangla-and-English
		clean_text = re.sub(r"[!\"”#$%&’()*+,-./:;<=>?@\[\]^_`{|}~–—\'।॥]+\ *", ' ', text)
		clean_text = re.sub(r"[-—]+\ *", ' ', clean_text)
		clean_text = re.sub(r"\n", '', clean_text)
		clean_text = " ".join(clean_text.split())

		# tokenize the cleaned text
		terms = clean_text.split(' ')
		# Remove stop words
		filtered_sentence = []
		for w in terms:
			if w not in stop_words:
				filtered_sentence.append(w)
		terms = filtered_sentence
		return terms

In [ ]:
tr_vectorizer = TfidfVectorizer(tokenizer=document_linearization().pre_process,stop_words=None, 
                             use_idf=True, 
                             smooth_idf=True,max_df=0.9)
tr_vectorizer.fit(tr_docs)
# The below tfidf_matix has the TF-IDF values of all the documents in the corpus. This is a big sparse doc-term matrix.
tr_tfidf_matrix  = tr_vectorizer.transform(tr_docs)

In [ ]:
c_vectorizer = TfidfVectorizer(tokenizer=document_linearization().pre_process,stop_words=None, 
                             use_idf=True, 
                             smooth_idf=True,max_df=0.9)
c_vectorizer.fit(c_docs)
# The below tfidf_matix has the TF-IDF values of all the articles in the concept corpus. This is a big sparse matrix.
c_tfidf_matrix  = c_vectorizer.transform(c_docs)

In [ ]:
tr_doc_term_matrix = tr_tfidf_matrix.toarray()
""" The rows in tr_doc_term_matrix represents the documents. To find which document is referred to do tr_file_paths[row_number_of_tr_doc_term_matrix]
    The columns in tr_doc_term_matrix represents the terms of the vocabulary of the train dataset.
"""
c_doc_term_matrix = c_tfidf_matrix.toarray()
tr_doc_concept_matrix = np.zeros((len(tr_file_paths),c_doc_term_matrix.shape[0]))

In [ ]:
# Returns a list of all the unique words in the vocabulary of Train Dataset.
tr_feature_names = tr_vectorizer.get_feature_names()

In [ ]:
# Returns a list of all the unique words in the vocabulary of Concept Dataset.
c_feature_names = c_vectorizer.get_feature_names()

In [ ]:
#@title For travel sentence embeddings
# bert_sent_path = '/content/drive/MyDrive/New_Work/IndicBert_Exp/travdoc_sentESA.pkl'
# esa_sent_path = '/content/drive/MyDrive/New_Work/IndicBert_Exp/TravLabse.pkl'

In [ ]:
#@title For health sentence embeddings
bert_sent_path = '/content/drive/MyDrive/New_Work/IndicBert_Exp/Healthdoc_sentESA.pkl'
esa_sent_path = '/content/drive/MyDrive/New_Work/IndicBert_Exp/LaBSE_Exp/LaBSE_Embeddings/HealthLabse.pkl'

In [ ]:
with open(bert_sent_path, 'rb') as f:
    doc_sentence_ESA= pickle.load(f)

In [ ]:
with open(esa_sent_path, 'rb') as f:
  doc_sentence_BERT = pickle.load(f)

In [ ]:
def document_wt(list1, list2):
    """
    The goal of Document weight function is to assign higher weight
    to documents containing more words from the query than documents having fewer query words
    :param list1: list of words in query after pre-processing the query.
    :param list2: list of words in the document after pre-processing.
    :return: Document-Query Overlap score in the range of [0,1].
    """
    s1 = set(list1)
    s2 = set(list2)
    intersection = float(len(s1.intersection(s2)))
    query_wt = float(len(s1))
    try:
        return intersection / query_wt
    except:
        return 0

In [ ]:
query_list = ["কোভিড মহামারীর দ্বিতীয় ঢেউয়ে আক্রান্তদের মধ্যে নানান জটিলতা"]

In [ ]:
doc_args= defaultdict(lambda: 0)

In [ ]:
tr_vectorizer1 = TfidfVectorizer(tokenizer=document_linearization().pre_process1,stop_words=None, 
                             use_idf=True, 
                             smooth_idf=True,max_df=0.9)
tr_vectorizer1.fit(tr_docs)
# The below tfidf_matix has the TF-IDF values of all the documents in the corpus. This is a big sparse doc-term matrix.
tr_tfidf_matrix1  = tr_vectorizer1.transform(tr_docs)

for  ind, query in enumerate(query_list):
  qrs=[query]
  qvecs = tr_vectorizer1.transform(qrs)
  q_idx = []
  for v in qvecs:
    q_idx.append(v.indices)
  for i in range(0,len(qrs)):
    A=sklearn.metrics.pairwise.cosine_similarity(qvecs,tr_tfidf_matrix1)
    row_max = A.max(axis=1)
    A = A / row_max[:, np.newaxis]
    # B = np.zeros(shape=A.shape)
    # for j in range(0, tr_tfidf_matrix.shape[0]):
    #   B[i][j] = document_wt([tr_feature_names[x] for x in q_idx[i]],[tr_feature_names[x] for x in tr_tfidf_matrix[j, :].nonzero()[1]])
    # for i in range(0, len(qrs)):
    #   for j in range(0, tr_tfidf_matrix.shape[0]):
    #     try:
    #       A[i][j] = statistics.harmonic_mean([A[i][j], B[i][j]])
    #     except:
    #       A[i][j] = 0 
    retrieved_docs=[]
    matching_scores=[]
    for i in range(0,len(qrs)):
      x=np.argsort(A[i]).T[::-1][:150]
      j=0
      for y in x:
        if A[i][y]>0:
          matching_scores.append(round(A[i][y],4))
          retrieved_docs.append(int(tr_file_paths[y].split('/')[-1].replace('.txt','')))
          j+=1
          doc_args[int(tr_file_paths[y].split('/')[-1].replace('.txt',''))]=dict()
          tfidf= defaultdict(lambda: 0)
          tfidf['tf-idf']=round(A[i][y],4)
          doc_args[int(tr_file_paths[y].split('/')[-1].replace('.txt',''))]=tfidf
    print("Query: ",qrs[i],"\ntf-idf: ",retrieved_docs[0:10])

In [ ]:
print("No. of document retrieved by tf-idf: ",len(retrieved_docs))

In [ ]:
matching_scores

Find the BERT Representation of the query.


In [ ]:
qbert = model.encode(query_list)
qbert = np.squeeze(qbert)

Find the “best” k(=3) sentence in a document which provides a good proxy for document relevance.

In [ ]:
for did in doc_args.keys():
  best_sents = []
  for sent, sent_vec in doc_sentence_BERT[did]:
    bert_score = np.dot(sent_vec,qbert)/(norm(sent_vec)*norm(qbert))
    best_sents.append(tuple((sent,bert_score)))
  best_sents = sorted(best_sents, key=lambda t: t[1], reverse=True)
  while(len(best_sents)<3):
    best_sents.append(tuple(("",0)))
  temp_dict=doc_args[did]
  temp_dict['S1']=best_sents[0][0]
  temp_dict['S1_bert'] = best_sents[0][1]
  temp_dict['S2']=best_sents[1][0]
  temp_dict['S2_bert'] = best_sents[0][1]
  temp_dict['S3']=best_sents[2][0]
  temp_dict['S3_bert'] = best_sents[2][1]
  doc_args[did] = temp_dict

Find the w_i's using nautral concepts described by humans themselves (by way of ESA).

In [ ]:
qrs = []
for q in query_list:
  q=normalizer.normalize(q)
  qrs.append(q)
qvecs=tr_vectorizer.transform(qrs)
Q=qvecs.toarray()
q_concept_matrix = np.zeros((len(qrs),c_doc_term_matrix.shape[0]))
for i in range (0,len(qrs)):
  q_feature_index = qvecs [i,:].nonzero()[1]
  q_tfidf_scores = zip(q_feature_index, [qvecs [i, x] for x in q_feature_index])
  tfidf_vec = np.zeros((c_doc_term_matrix.shape[0]))
  # w represents the term 't'
  # s represents the tfidf score of the term 't' in document 'd'.
  for w, s in [(tr_feature_names[i], s) for (i, s) in q_tfidf_scores]:
    try:
      c_word_index = c_vectorizer.vocabulary_[w]
      # All the documents in the Concept_Dataset is considered to be a wikipedia concept.
      temp_vec = np.zeros((c_doc_term_matrix.shape[0]))
      for j in range(0,c_doc_term_matrix.shape[0]):
        temp_vec[j] =  c_doc_term_matrix[j][c_word_index]* s
      tfidf_vec = tfidf_vec + temp_vec
    except:
      continue
  q_concept_matrix[i] = tfidf_vec 
  qESA=scipy.sparse.csr_matrix(q_concept_matrix)

In [ ]:
def get_concept_representation(word, tf_idf_score, concept_vec_size):
  concept_vec = np.zeros((concept_vec_size))
  try:
    c_word_index = c_vectorizer.vocabulary_[w]
    for j in range(0,c_doc_term_matrix.shape[0]):
      concept_vec[j] =  c_doc_term_matrix[j][c_word_index] * tf_idf_score
  except:
    return scipy.sparse.csr_matrix(np.zeros((concept_vec_size)))
  return scipy.sparse.csr_matrix(concept_vec) 

In [ ]:
#@title Execute this cell if you dont want sentence explanation with ESA
for did in doc_args.keys():
  temp_dict=doc_args[did]
  for idx, tup in enumerate(doc_sentence_ESA[did]):
    if tup[0] == doc_args[did]['S1']:
      S1_ESA = scipy.sparse.csr_matrix(doc_sentence_ESA[did][idx][1])
      w1 = sklearn.metrics.pairwise.cosine_similarity(qESA,S1_ESA).item()
      temp_dict['w1']=w1
    if tup[0] == doc_args[did]['S2']:
      S2_ESA = scipy.sparse.csr_matrix(doc_sentence_ESA[did][idx][1])
      w2 = sklearn.metrics.pairwise.cosine_similarity(qESA,S2_ESA).item()
      temp_dict['w2']=w2
    if tup[0] == doc_args[did]['S3']:
      S3_ESA = scipy.sparse.csr_matrix(doc_sentence_ESA[did][idx][1])
      w3 = sklearn.metrics.pairwise.cosine_similarity(qESA,S3_ESA).item()
      temp_dict['w3']=w3
  doc_args[did] = temp_dict

In [ ]:
#@title Execute this cell if you want sentence explanation with ESA
for did in doc_args.keys():
  temp_dict=doc_args[did]
  for idx, tup in enumerate(doc_sentence_ESA[did]):
    try:
      if tup[0] == doc_args[did]['S1']:
        S1_ESA = scipy.sparse.csr_matrix(doc_sentence_ESA[did][idx][1])
        w1 = sklearn.metrics.pairwise.cosine_similarity(qESA,S1_ESA).item()
        temp_dict['w1']=w1
        i = tr_dids.index(did)
        tr_feature_index = tr_tfidf_matrix [i,:].nonzero()[1]
        tr_tfidf_scores = zip(tr_feature_index, [tr_tfidf_matrix [i, x] for x in tr_feature_index])
        tfidf_vec = np.zeros((c_doc_term_matrix.shape[0]))
        doc_token_tfidf=dict()
        for w, s in [(tr_feature_names[i], s) for (i, s) in tr_tfidf_scores]:
          doc_token_tfidf[w]=s
        word_rel_score=[]
        max_word_score = 0.0
        for w in document_linearization().pre_process(doc_args[did]['S1']):
          try:
            s = doc_token_tfidf[w]
            word_score = round(sklearn.metrics.pairwise.cosine_similarity(get_concept_representation(w,s,c_doc_term_matrix.shape[0]),qESA)[0][0],4)
            max_word_score = max(max_word_score,word_score)
            word_rel_score.append([w,word_score])
          except:
            continue
        explainable_terms = []
        for word_index in range (0,len(word_rel_score)):
          word_rel_score[word_index][1] = round(word_rel_score[word_index][1]/max_word_score,4)
          if word_rel_score[word_index][1]> 0.5*max_word_score:
            explainable_terms.append(tuple(word_rel_score[word_index]))
        explainable_terms = sorted(explainable_terms, key=lambda t: t[1], reverse=True)
        temp_dict['S1_ESA']=list(explainable_terms)
    except:
      pass
    try:
      if tup[0] == doc_args[did]['S2']:
        S2_ESA = scipy.sparse.csr_matrix(doc_sentence_ESA[did][idx][1])
        w2 = sklearn.metrics.pairwise.cosine_similarity(qESA,S2_ESA).item()
        temp_dict['w2']=w2
        i = tr_dids.index(did)
        tr_feature_index = tr_tfidf_matrix [i,:].nonzero()[1]
        tr_tfidf_scores = zip(tr_feature_index, [tr_tfidf_matrix [i, x] for x in tr_feature_index])
        tfidf_vec = np.zeros((c_doc_term_matrix.shape[0]))
        doc_token_tfidf=dict()
        for w, s in [(tr_feature_names[i], s) for (i, s) in tr_tfidf_scores]:
          doc_token_tfidf[w]=s
        word_rel_score=[]
        max_word_score = 0.0
        for w in document_linearization().pre_process(doc_args[did]['S2']):
          try:
            s = doc_token_tfidf[w]
            word_score = round(sklearn.metrics.pairwise.cosine_similarity(get_concept_representation(w,s,c_doc_term_matrix.shape[0]),qESA)[0][0],4)
            max_word_score = max(max_word_score,word_score)
            word_rel_score.append([w,word_score])
          except:
            continue
        explainable_terms = []
        for word_index in range (0,len(word_rel_score)):
          word_rel_score[word_index][1] = round(word_rel_score[word_index][1]/max_word_score,4)
          if word_rel_score[word_index][1]> 0.5*max_word_score:
            explainable_terms.append(tuple(word_rel_score[word_index]))
        explainable_terms = sorted(explainable_terms, key=lambda t: t[1], reverse=True)
        temp_dict['S2_ESA']=list(explainable_terms)
    except:
      pass
    try:
      if tup[0] == doc_args[did]['S3']:
        S3_ESA = scipy.sparse.csr_matrix(doc_sentence_ESA[did][idx][1])
        w3 = sklearn.metrics.pairwise.cosine_similarity(qESA,S3_ESA).item()
        temp_dict['w3']=w3
        i = tr_dids.index(did)
        tr_feature_index = tr_tfidf_matrix [i,:].nonzero()[1]
        tr_tfidf_scores = zip(tr_feature_index, [tr_tfidf_matrix [i, x] for x in tr_feature_index])
        tfidf_vec = np.zeros((c_doc_term_matrix.shape[0]))
        doc_token_tfidf=dict()
        for w, s in [(tr_feature_names[i], s) for (i, s) in tr_tfidf_scores]:
          doc_token_tfidf[w]=s
        word_rel_score=[]
        max_word_score = 0.0
        for w in document_linearization().pre_process(doc_args[did]['S3']):
          try:
            s = doc_token_tfidf[w]
            word_score = round(sklearn.metrics.pairwise.cosine_similarity(get_concept_representation(w,s,c_doc_term_matrix.shape[0]),qESA)[0][0],4)
            max_word_score = max(max_word_score,word_score)
            word_rel_score.append([w,word_score])
          except:
            continue
        explainable_terms = []
        for word_index in range (0,len(word_rel_score)):
          word_rel_score[word_index][1] = round(word_rel_score[word_index][1]/max_word_score,4)
          if word_rel_score[word_index][1]> 0.5*max_word_score:
            explainable_terms.append(tuple(word_rel_score[word_index]))
        explainable_terms = sorted(explainable_terms, key=lambda t: t[1], reverse=True)
        temp_dict['S3_ESA']=list(explainable_terms)
    except:
      pass
  doc_args[did] = temp_dict

In [ ]:
S2_ESA.shape

Calcualate the final matching score of the document to the query.

In [ ]:
def calc_q_doc_bert(sdoc, alpha, w1, w2, w3, S1_bert, S2_bert, S3_bert, weight):
    if weight == "equal":
      w1=1
      w2=1
      w3=1
    weight_list = [w1, w2, w3]
    bert_sent_score = [S1_bert, S2_bert, S3_bert]
    sum_score = 0
    for s, w in zip(weight_list, bert_sent_score):
      sum_score += s * w
    doc_score = alpha * sdoc + (1.0 - alpha) * sum_score
    return doc_score

In [ ]:
retrieved_rank = []
try:
  for did in doc_args.keys():
    final_doc_score = calc_q_doc_bert(sdoc = doc_args[did]['tf-idf'],alpha = 0.5, w1 = doc_args[did]['w1'], w2 = doc_args[did]['w2'], w3 = doc_args[did]['w3'],S1_bert = doc_args[did]['S1_bert'], S2_bert = doc_args[did]['S2_bert'], S3_bert = doc_args[did]['S3_bert'], weight="not equal")
    retrieved_rank.append(tuple((did,final_doc_score)))
except:
  print("Some Error",did)
retrieved_rank = sorted(retrieved_rank, key=lambda t: t[1], reverse=True)
# print(retrieved_rank[:10])
print("ESA Weights: ",list(zip(*retrieved_rank[:10]))[0])

In [ ]:
retrieved_rank = []
try:
  for did in doc_args.keys():
    final_doc_score = calc_q_doc_bert(sdoc = doc_args[did]['tf-idf'],alpha = 0.5, w1 = doc_args[did]['w1'], w2 = doc_args[did]['w2'], w3 = doc_args[did]['w3'],S1_bert = doc_args[did]['S1_bert'], S2_bert = doc_args[did]['S2_bert'], S3_bert = doc_args[did]['S3_bert'], weight="equal")
    retrieved_rank.append(tuple((did,final_doc_score)))
except:
  print("Some Error",did)
retrieved_rank = sorted(retrieved_rank, key=lambda t: t[1], reverse=True)
# print(retrieved_rank[:10])
print("Equal Weights: ",list(zip(*retrieved_rank[:10]))[0])

In [ ]:
for did,score in retrieved_rank[0:3]:
  print("Doc id: ",did)
  print("S1: ",doc_args[did]['S1'])
  print("S2: ",doc_args[did]['S2'])
  print("S3: ",doc_args[did]['S3'])
  print()